In [1]:
from strategy.weekly_price import WeeklyPrice
from strategy.quarterly_financial_categorical import QuarterlyFinancialCategorical
from backtester.backtester import Backtester as b
from database.market import Market
from preprocessor.model_preprocessor import ModelPreprocessor as mp
from modeler.modeler import Modeler as m
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np
import math

In [2]:
## Loading Constants
market = Market()
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
strat = WeeklyPrice()
qfc = QuarterlyFinancialCategorical()
year = datetime.now().year
week = datetime.now().isocalendar()[1]

In [3]:
week

31

In [4]:
categories = qfc.retrieve_data("portfolio_stock_category_sim")

In [5]:
categories.columns

Index(['_id', 'year', 'quarter', 'ticker', 'prediction'], dtype='object')

In [6]:
strat.db.connect()
sim = strat.db.retrieve_data("portfolio_weekly_predictions")
strat.db.close()

In [7]:
market.connect()
price = market.retrieve_data("portfolio_prices").drop("_id",axis=1)
market.close()

In [8]:
price.sort_values("date")

,date,pdr,ticker,tiingo,finnhub,adjclose,week,quarter,year
0,2019-06-03,149.246216,MMM,149.187445,164.29,154.241220,23.0,2.0,2019.0
199792,2019-06-03,12.857295,RF,12.863179,14.33,13.350158,23.0,2.0,2019.0
29677,2019-06-03,20.297375,BKR,20.297066,22.04,20.878147,23.0,2.0,2019.0
199289,2019-06-03,303.950012,REGN,303.950000,302.06,303.320004,23.0,2.0,2019.0
198786,2019-06-03,60.915817,REG,60.961144,65.85,62.575654,23.0,2.0,2019.0
...,...,...,...,...,...,...,...,...,...
270241,2021-07-30,NaN,ENPH,189.600000,NaN,189.600000,30.0,3.0,2021.0
270240,2021-07-30,NaN,EMR,100.890000,NaN,100.890000,30.0,3.0,2021.0
270239,2021-07-30,NaN,EMN,112.720000,NaN,112.720000,30.0,3.0,2021.0
270252,2021-07-30,NaN,EXC,46.800000,NaN,46.800000,30.0,3.0,2021.0


In [9]:
price["year"] = [x.year for x in price["date"]]
price["quarter"] = [x.quarter for x in price["date"]]
price["week"] = [x.week for x in price["date"]]

In [10]:
sim["week"].max()

30

In [11]:
price["week"].max()

53

In [12]:
sim = sim.merge(price[["date","year","quarter","week","ticker","adjclose"]],on=["year","quarter","week","ticker"],how="left")
sim["weekly_predicted_delta"]  = (sim["weekly_price_categorical_regression_prediction"] - sim["adjclose"]) / sim["adjclose"]

In [13]:
sim["week"].max()

30

In [14]:
classification = False
trade_signal_score = 0.81

In [15]:
sim = sim[sim["date"] > datetime(2021,7,29)]

In [16]:
sim

,_id,year,quarter,week,ticker,weekly_price_categorical_regression_prediction,weekly_price_categorical_regression_score,weekly_price_categorical_classification_prediction,weekly_price_categorical_classification_score,date,adjclose,weekly_predicted_delta
28811,6108105f16fcbf1a9c8c516a,2021,3,30,AAL,22.545681,0.993949,1,0.600000,2021-07-30,20.38,0.106265
28815,6108105f16fcbf1a9c8c516b,2021,3,30,AOS,71.089462,0.993949,1,0.600000,2021-07-30,70.33,0.010799
28819,6108105f16fcbf1a9c8c516c,2021,3,30,CSX,33.525299,0.993949,1,0.600000,2021-07-30,32.32,0.037293
28823,6108105f16fcbf1a9c8c516d,2021,3,30,EMR,99.556946,0.993949,1,0.600000,2021-07-30,100.89,-0.013213
28827,6108105f16fcbf1a9c8c516e,2021,3,30,EXPD,120.205338,0.993949,1,0.600000,2021-07-30,128.25,-0.062726
...,...,...,...,...,...,...,...,...,...,...,...,...
30063,6108105f16fcbf1a9c8c52a3,2021,3,30,EA,141.810349,0.992990,0,0.567568,2021-07-30,143.96,-0.014932
30067,6108105f16fcbf1a9c8c52a4,2021,3,30,FB,309.085785,0.992990,1,0.567568,2021-07-30,356.30,-0.132513
30071,6108105f16fcbf1a9c8c52a5,2021,3,30,TMUS,141.694183,0.992990,1,0.567568,2021-07-30,144.02,-0.016149
30075,6108105f16fcbf1a9c8c52a6,2021,3,30,PXD,138.752441,0.939811,0,0.692308,2021-07-30,145.37,-0.045522


In [17]:
for col in sim.columns:
    if 'delta' in col:
        sim["{}_abs".format(col)] = [abs(x) for x in sim[col]]
        filtered_sim = sim[(sim["{}_abs".format(col)] <= 1)]
for col in sim.columns:
    if "regression_score" in col:
        filtered_sim = filtered_sim[filtered_sim[col] >= trade_signal_score]
filtered_sim = filtered_sim[(filtered_sim["weekly_predicted_delta"] >= 0)]
if classification:
    for col in sim.columns:
        if "classification_score" in col:
            filtered_sim = filtered_sim[filtered_sim[col] >= trade_signal_score]
        if "classification_prediction" in col:
            filtered_sim = filtered_sim[filtered_sim[col] == 1]

In [18]:
filtered_sim = filtered_sim.groupby(["year","quarter","week","date","ticker"]).mean().reset_index()

In [19]:
 report = filtered_sim.sort_values("weekly_predicted_delta",ascending=False).head(4).merge(categories,on=["year","quarter","ticker"])

In [20]:
seats = 1.0
capital = 500.0

In [21]:
report

,year,quarter,week,date,ticker,weekly_price_categorical_regression_prediction,weekly_price_categorical_regression_score,weekly_price_categorical_classification_prediction,weekly_price_categorical_classification_score,adjclose,weekly_predicted_delta,weekly_predicted_delta_abs,_id,prediction
0,2021,3,30,2021-07-30,AAL,22.545681,0.993949,1,0.600000,20.38,0.106265,0.106265,60e2cf7e1f7f6d37014ad641,100Industrials
1,2021,3,30,2021-07-30,NWL,27.299173,0.991296,1,0.640741,24.75,0.102997,0.102997,60e2cf7e1f7f6d37014ad78f,100Consumer Discretionary
2,2021,3,30,2021-07-30,HPE,15.778098,0.991714,1,0.662791,14.50,0.088145,0.088145,60e2cf7e1f7f6d37014ad718,100Information Technology
3,2021,3,30,2021-07-30,WY,36.537300,0.991637,1,0.577778,33.73,0.083229,0.083229,60e2cf7e1f7f6d37014ad825,100Real Estate


In [22]:
trade = report[["year","quarter","week","date","ticker","prediction","weekly_predicted_delta","adjclose","weekly_price_categorical_regression_prediction","weekly_price_categorical_regression_score"]]

In [23]:
trade["sell_price"] = trade["weekly_price_categorical_regression_prediction"]
trade["shares"] = (capital/seats/trade["adjclose"])

In [24]:
final = trade[["year","quarter","week","date","ticker","prediction","adjclose","sell_price","shares","weekly_price_categorical_regression_score"]]

In [25]:
final

,year,quarter,week,date,ticker,prediction,adjclose,sell_price,shares,weekly_price_categorical_regression_score
0,2021,3,30,2021-07-30,AAL,100Industrials,20.38,22.545681,24.533857,0.993949
1,2021,3,30,2021-07-30,NWL,100Consumer Discretionary,24.75,27.299173,20.202020,0.991296
2,2021,3,30,2021-07-30,HPE,100Information Technology,14.50,15.778098,34.482759,0.991714
3,2021,3,30,2021-07-30,WY,100Real Estate,33.73,36.537300,14.823599,0.991637


In [26]:
filtered_sim[filtered_sim["ticker"].isin(["HPQ","GPS","NLOK"])]

,year,quarter,week,date,ticker,weekly_price_categorical_regression_prediction,weekly_price_categorical_regression_score,weekly_price_categorical_classification_prediction,weekly_price_categorical_classification_score,adjclose,weekly_predicted_delta,weekly_predicted_delta_abs
61,2021,3,30,2021-07-30,GPS,30.666376,0.991296,1,0.640741,29.17,0.051298,0.051298
65,2021,3,30,2021-07-30,HPQ,29.496490,0.991714,1,0.662791,28.87,0.021700,0.021700


In [33]:
strat.db.connect()
strat.db.store_data("portfolio_trades",final)
strat.db.close()

In [34]:
strat.db.connect()
historical = strat.db.retrieve_data("portfolio_trades")
strat.db.close()

In [36]:
sim[(sim["ticker"].isin(["HPQ","NLOK","GPS"])) & (sim["week"]==28)]

,_id,year,quarter,week,ticker,weekly_price_categorical_regression_prediction,weekly_price_categorical_regression_score,weekly_price_categorical_classification_prediction,weekly_price_categorical_classification_score,date,adjclose,weekly_predicted_delta,weekly_predicted_delta_abs
22950,60edbc8eb2afe8ae6b05f72c,2021,3,28,HPQ,29.496490,0.991714,1,0.662791,2021-07-16,27.52,0.071820,0.071820
23643,60edbc8eb2afe8ae6b05f7c6,2021,3,28,GPS,35.250607,0.991296,1,0.640741,2021-07-16,28.32,0.244725,0.244725
26002,60f539dc8e1a8b56b83c00a0,2021,3,28,HPQ,29.496490,0.991714,1,0.662791,2021-07-16,27.52,0.071820,0.071820
26007,60f539dc8e1a8b56b83c00a1,2021,3,28,HPQ,29.496490,0.991714,1,0.662791,2021-07-16,27.52,0.071820,0.071820
26772,60f539dc8e1a8b56b83c013a,2021,3,28,GPS,35.250607,0.991296,1,0.640741,2021-07-16,28.32,0.244725,0.244725
26777,60f539dc8e1a8b56b83c013b,2021,3,28,GPS,31.677729,0.991296,1,0.640741,2021-07-16,28.32,0.118564,0.118564
